# Probleme durch Überanpassung
Durch Hinzunahme weiterer Parameter des Modells (beispielsweise durch die Erhöhung des Polynomgrades) scheint die Anpassung zunehmend besser zu werden. In diesem Abschnitt sollen Probleme aufgezeigt werden, die damit einhergehen.

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import random
from sklearn.model_selection import train_test_split

from draw import draw_regression, R_squared
from ipywidgets import interact, IntSlider

## Inhaltsverzeichnis
- [Trainings- und Testdaten](#Trainings--und-Testdaten)
- [Schätzung der Parameter](#Schätzung-der-Parameter)

## Trainings- und Testdaten
Zunächst werden Trainings- und Testdaten etwa im Verhältnis 3:1 erstellt. Sie orientieren sich wie zuvor an einer Funktion quadratischer Ordnung.

In [ ]:
np.random.seed(1)
random.seed(1)

df = pd.DataFrame({
    'x': (random.random() * 10 - 4 for _ in range(200))
})
df['y'] = 5 - 2 * df['x'] + df['x'] ** 2
df['y'] += np.random.normal(loc=0, scale=3, size=200)
df['c'] = random.choices(['train', 'test'], k=200, weights=[0.75, 0.25])

px.scatter(df, x='x', y='y', color='c')

In [ ]:
train, test = df[df['c'] == 'train'], df[df['c'] == 'test']
len(train), len(test)

## Schätzung der Parameter
Erhöhen Sie langsam die Anzahl der verwendeten Parameter und beobachten Sie die dabei, wie gut die Ausgleichskurve jeweils die Trainings- und Testobjekte abbildet.

In [ ]:
def est(n):
    A = np.zeros((n, n))
    for row in range(n):
        for col in range(n):
            A[row,col] = (train['x'] ** (row+col)).sum()

    b = np.zeros(n)
    for row in range(n):
        b[row] = ((train['x'] ** row) * train['y']).sum()

    return np.linalg.solve(A, b)

@interact(n=IntSlider(2, 1, 50, 1))
def _(n):
    x = est(n)

    fig = px.scatter(df, x='x', y='y', color='c', range_y=[-5, 35])
    draw_regression(fig, *x)

    return fig

Die folgende Grafik zeigt dagegen das (durchschnittliche) quadratische Residuum in Abhängigkeit der Anzahl der Parameter für die Trainings- und die Testmenge.

In [ ]:
ns = [1, 2, 3, 4, 5, *range(10, 31, 2)]
r2tr, r2te = [], []

for n in ns:
    x = est(n)
    r2tr.append(R_squared(train['x'], train['y'], *x) / len(train))
    r2te.append(R_squared(test['x'], test['y'], *x) / len(test))

dfr = pd.DataFrame({
    'n': ns,
    'R^2 (train)': r2tr,
    'R^2 (test)': r2te
})

px.line(dfr, x='n', y=['R^2 (train)', 'R^2 (test)'])

Während die Residuen der Trainingsobjekte sinken und diese Menge damit scheinbar immer besser repräsentiert wird, steigen die Residuen der Testmenge. Zurückzuführen ist dies auf eine Überanpassung an die Trainingsmenge.